In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import pandas
import numpy as np
import pickle
from pts_loader import load

In [2]:
tar_res = 224

In [3]:
set_types = ['trainset', 'testset']

In [4]:
db_all = {}
for types in set_types:
    print(types)
    db_path = 'G:/datasets/facial_landmarks/'+types+'/'
    if types == 'trainset':
        db_name = ['afw','helen','lfpw']
        img_sub = ['.jpg','.jpg','.png']
    elif types == 'testset':
        db_name = ['300w','helen','ibug','lfpw','cofw']
        img_sub = ['.png','.jpg','.jpg','.png','.jpg']
    fd_imgs=[]
    fd_pts=[]
    fd_dataset = []
    fd_fn = []
    for dn, sub in zip(db_name, img_sub):
        print('loading...', dn)
        os.listdir(db_path+dn+'/');#print(os.listdir(db_path+dn+'/'))

        db_folder = db_path+dn+'/'
        fns = [f.split('.')[0] for f in os.listdir(db_folder) if os.path.isfile(os.path.join(db_folder, f))];#print(list(set(fns)))
        fns = list(set(fns))
        fns.sort()

        # f = '2437602091_1'
        for f in fns:
    #         print(f)
            #f = fns[1];
            image_path = db_folder + f
            #load image and pts
            img = cv2.imread(image_path + sub, 1)
            pts = np.ndarray.astype(np.round(np.asarray(load(image_path + '.pts'))), dtype = 'int32')
            # get face location
            fl_x_min = np.min(pts[:,0])
            fl_x_max = np.max(pts[:,0])
            fl_y_min = np.min(pts[:,1])
            fl_y_max = np.max(pts[:,1])
            # 'landmark out of boundary' pads with zeros
    #         if (fl_x_min < 0) or (fl_y_min < 0) or (fl_y_max > (img.shape[0]-1)) or (fl_x_max > (img.shape[1]-1)):

    #             print ('landmark out of boundary')
            # clac bounding box
            bx_size = int(1.3*max((fl_y_max - fl_y_min), (fl_x_max - fl_x_min)))
            bx_center = ((fl_x_max + fl_x_min)/2,(fl_y_max + fl_y_min)/2); #print(bx_center)
            bx_lt =  np.ndarray.astype(np.round(bx_center - np.asarray(bx_size/2)), dtype = 'int32'); #print("bx_lt",bx_lt)
            bx_rb =  bx_lt+bx_size; #print("bx_rb",bx_rb)
            # create space 
            crop_image = np.zeros((bx_size,bx_size,3),dtype=np.uint8)

            img_t = 0
            img_l = 0
            img_b = bx_size
            img_r = bx_size

            if bx_lt[0] < 0:
                img_l = abs(bx_lt[0])
                pts = pts + [abs(bx_lt[0]),0]
                bx_lt[0] = 0

            if bx_lt[1] < 0:
                img_t = abs(bx_lt[1])
                pts = pts + [0,abs(bx_lt[1])]
                bx_lt[1] = 0

            if bx_rb[0] >= img.shape[1]:
                img_r = bx_size - (bx_rb[0] - img.shape[1])
                bx_rb[0] = img.shape[1]

            if bx_rb[1] >= img.shape[0]:
                img_b = bx_size - (bx_rb[1] - img.shape[0])
                bx_rb[1] = img.shape[0]

    #         print(img_l,img_t)
    #         print(img_r,img_b)
    #         print("bx_lt",bx_lt)
    #         print("bx_rb",bx_rb)
            # crop image
            crop_image[img_t:img_b,img_l:img_r]  = crop_image[img_t:img_b,img_l:img_r] + img[bx_lt[1]:(bx_rb[1]),
                                                                                             bx_lt[0]:(bx_rb[0]),:]
            shift_pts = pts - bx_lt
            resize_ratio = tar_res/crop_image.shape[1]

            # resized dataset
            res_img = cv2.resize(crop_image, dsize=(tar_res, tar_res), interpolation=cv2.INTER_CUBIC)
            resized_pts = np.ndarray.astype(np.round(shift_pts*resize_ratio), dtype = 'int32')

            fd_imgs.append(np.asarray(res_img))
            fd_pts.append(np.asarray(resized_pts))
            fd_dataset.append(dn)
            fd_fn.append(dn + '_'+ f +'.png')


            #plot and check
            # plt.imshow(res_img)
            # plt.show()

            fig_img = res_img.copy()
    #         for i in range(resized_pts.shape[0]):
    #             #print(pts.iloc[i,0],pts.iloc[i,1])
    #             cv2.circle(fig_img,(resized_pts[i,0],
#                                     resized_pts[i,1]), 2, (255,0,0), -1)
#             plt.imshow(fig_img)
#             plt.imsave(db_path+'crop_img/' + dn + '_'+ f +'.png', fig_img, format="png")
#             plt.close().png
        #     plt.show()
            cv2.imwrite(db_path+'crop_img/' + dn + '_'+ f +'.png', res_img) 
    db = {}
    db['img'] = np.asarray(fd_imgs)
    db['pts'] = np.asarray(fd_pts)
    db['dataset'] = np.asarray(fd_dataset)
    db['file_name'] = np.asarray(fd_fn)
    db_all[types] = db

trainset
loading... afw
loading... helen
loading... lfpw
testset
loading... 300w
loading... helen
loading... ibug
loading... lfpw
loading... cofw


In [5]:
print(db_all.keys())
print(db_all['trainset'].keys())
print(db_all['testset'].keys())
print(db_all['trainset']['pts'].shape)
print(db_all['trainset']['img'].shape)
print(db_all['testset']['pts'].shape)
print(db_all['testset']['img'].shape)

dict_keys(['testset', 'trainset'])
dict_keys(['img', 'file_name', 'dataset', 'pts'])
dict_keys(['img', 'file_name', 'dataset', 'pts'])
(3148, 68, 2)
(3148, 224, 224, 3)
(1796, 68, 2)
(1796, 224, 224, 3)


In [6]:
with open('G:/datasets/facial_landmarks/combined_new.pickle', 'wb') as handle:
    pickle.dump(db_all, handle)

# image and points to pickle

In [7]:
# for dn in db_name:
#     os.listdir(db_path+dn+'/');print(os.listdir(db_path+dn+'/'))
#     db = {}
#     db_imgs={}
#     db_pts={}
#     for t in os.listdir(db_path+dn+'/'):
#         db_folder = db_path+dn+'/'+t+'/'
#         fns = [f.split('.')[0] for f in os.listdir(db_folder) if os.path.isfile(os.path.join(db_folder, f))];#print(list(set(fns)))
#         fns = list(set(fns))
#         fns.sort()

#         fd_imgs=[]
#         fd_pts=[]
#         for f in fns:
#             image_path = db_folder + f
#             img = cv2.imread(image_path +'.png', 1)
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#             ratio = img.shape[0]/tar_res
#             img = cv2.resize(img,(int(img.shape[0]/ratio),int(img.shape[1]/ratio)),interpolation=cv2.INTER_CUBIC)
#             fd_imgs.append(np.asarray(img))
#             pts = pandas.read_csv(image_path +'.txt', header=None);# print(pts)
#             fd_pts.append(np.around(pts.values/ratio).astype(np.int32))
            
#         # print
# #         fig_img = fd_imgs[0].copy()
# #         for i in range(fd_pts[0].shape[0]):
# #             #print(pts.iloc[i,0],pts.iloc[i,1])
# #             cv2.circle(fig_img,(fd_pts[0][i,0],fd_pts[0][i,1]), 2, (255,0,0), -1)
# #         plt.imshow(fig_img)
# #         plt.show()

#         db_imgs[t]= np.asarray(fd_imgs)
#         db_pts[t] = np.asarray(fd_pts)

#     # check shape
#     print(db_imgs.keys(),db_pts.keys())
#     for k in list(db_imgs.keys()):
#         print(db_imgs[k].shape, db_pts[k].shape)
#     # save to db
#     db['img']= db_imgs
#     db['pts']= db_pts
    
#     with open('./dataset/'+dn+'.pickle', 'wb') as handle:
#         pickle.dump(db, handle)